In [1]:
# chose the current file directory as the working directory
import os 
os.chdir("/teamspace/studios/this_studio/rubikscubesolver")


In [2]:
from tqdm import tqdm

import wandb  # for logging
import time
from dataclasses import dataclass

import jax
import jax.numpy as jnp
import flax.nnx as nnx

import optax

from rubiktransformer.models import RubikTransformer, PolicyModel
import rubiktransformer.dataset as dataset
from rubiktransformer.trainer import train
from rubiktransformer.trainer import reshape_sample

cuda_plugin_extension is not found.


In [3]:
@dataclass
class Config:
    """Configuration class"""
    jax_key: jnp.ndarray = jax.random.PRNGKey(46)
    rngs = nnx.Rngs(45)
    batch_size: int = 128
    lr_1: float = 4e-3
    lr_2: float = 4e-3
    nb_games: int = 128 * 100
    len_seq: int = 15
    nb_step: int = 1000000
    log_every_step: int = 10
    log_eval_every_step: int = 10
    log_policy_reward_every_step: int = 10
    add_data_every_step: int = 500

config = Config()

# init wandb config
user = "forbu14"
project = "RubikTransformer"
display_name = "experiment_" + time.strftime("%Y%m%d-%H%M%S")

wandb.init(entity=user, project=project, name=display_name)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: forbu14. Use `wandb login --relogin` to force relogin


In [4]:
policy = PolicyModel(rngs=config.rngs)
transformer = RubikTransformer(rngs=config.rngs, causal=True)

scheduler = optax.linear_schedule(init_value=0., end_value=1., transition_steps=4000)

# init optimizer
optimizer_optaxworldmodel = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.lion(config.lr_1 / 100.),
   # optax.adamw(config.lr_1/10.),
    optax.scale_by_schedule(scheduler),
)

optimizer_worldmodel = nnx.Optimizer(transformer, optimizer_optaxworldmodel)
optimizer_policy = nnx.Optimizer(policy, optax.adam(config.lr_2))

# metrics
metrics_train = nnx.MultiMetric(
    loss=nnx.metrics.Average("loss"),
    loss_reward=nnx.metrics.Average("loss_reward"),
    loss_cross_entropy=nnx.metrics.Average("loss_cross_entropy"),
)

metrics_eval = nnx.MultiMetric(
    loss_eval=nnx.metrics.Average("loss_eval"),
    loss_reward_eval=nnx.metrics.Average("loss_reward_eval"),
    loss_cross_entropy_eval=nnx.metrics.Average("loss_cross_entropy_eval"),
)


In [5]:
# gather data from the environment
# init models and optimizers
env, buffer = dataset.init_env_buffer(sample_batch_size=config.batch_size)
env, buffer_eval = dataset.init_env_buffer(sample_batch_size=config.batch_size)


nb_games = config.nb_games
len_seq = config.len_seq

state_first = jnp.zeros((6, 3, 3))
state_next = jnp.zeros((len_seq, 6, 3, 3))
action = jnp.zeros((len_seq, 3))
action_proba = jnp.zeros((len_seq, 9))

# transform state to int8 type
state_first = state_first.astype(jnp.int8)
state_next = state_next.astype(jnp.int8)

# action to int32 type
action = action.astype(jnp.int32)

reward = jnp.zeros((len_seq))

jit_step = jax.jit(env.step)

buffer_list = buffer.init(
    {
        "state_first": state_first,
        "action": action,
        "reward": reward,
        "state_next": state_next,
        "action_pred": action_proba,
    }
)

buffer_list_eval = buffer_eval.init(
    {
        "state_first": state_first,
        "action": action,
        "reward": reward,
        "state_next": state_next,
        "action_pred": action_proba,
    }
)


In [6]:

def step_fn(state, key):
    """
    Simple step function
    We choose a random action
    """

    action = jax.random.randint(
        key=key,
        minval=env.action_spec.minimum,
        maxval=env.action_spec.maximum,
        shape=(3,),
    )

    new_state, timestep = jit_step(state, action)
    timestep.extras["action"] = action
    timestep.extras["action_pred"] = jnp.zeros((9,))

    return new_state, timestep

def run_n_steps(state, key, n):
    random_keys = jax.random.split(key, n)
    state, rollout = jax.lax.scan(step_fn, state, random_keys)

    return rollout

vmap_reset = jax.vmap(jax.jit(env.reset))
vmap_step = jax.vmap(run_n_steps, in_axes=(0, 0, None))

In [7]:
scale_factor = 0.01

def step_fn_proba_setup(state, data):
    """
    Simple step function
    We choose a random action
    """

    key1, key2 = data

    # we choose a probability distribution over the action
    action_proba_0 = jax.random.normal(key1, shape=(6,)) * scale_factor
    action_proba_1 = jax.random.normal(key2, shape=(3,)) * scale_factor

    # softmax over the action
    action_proba_0 = jax.nn.softmax(action_proba_0)
    action_proba_1 = jax.nn.softmax(action_proba_1)

    # sample the action from the probability distribution
    action_proba_0_value = jax.random.categorical(key1, action_proba_0)
    action_proba_1_value = jax.random.categorical(key2, action_proba_1)

    action = jnp.array([action_proba_0_value, jnp.array(0), action_proba_1_value])

    # concat 
    action_proba = jnp.concatenate([action_proba_0, action_proba_1])

    new_state, timestep = jit_step(state, action)
    timestep.extras["action"] = action
    timestep.extras["action_pred"] = action_proba

    return new_state, timestep

def run_n_steps_proba(state, key, n):
    random_keys = jax.random.split(key, (n, 2))
    state, rollout = jax.lax.scan(step_fn_proba_setup, state, random_keys)

    return rollout

vmap_reset = jax.vmap(jax.jit(env.reset))
vmap_step_proba = jax.vmap(run_n_steps_proba, in_axes=(0, 0, None))

In [8]:
nnx.display(transformer)

In [9]:
key, subkey = jax.random.split(config.jax_key)
config.jax_key = key

buffer, buffer_list = dataset.fast_gathering_data(
    env,
    vmap_reset,
    vmap_step,
    int(config.nb_games / 10),
    config.len_seq,
    buffer,
    buffer_list,
    subkey,
)

In [10]:
key, subkey = jax.random.split(config.jax_key)
config.jax_key = key

buffer, buffer_list = dataset.fast_gathering_data(
    env,
    vmap_reset,
    vmap_step_proba,
    int(config.nb_games / 10),
    config.len_seq,
    buffer,
    buffer_list,
    subkey,
)

In [11]:

sample = buffer.sample(buffer_list, subkey)
sample = reshape_sample(sample)

In [28]:
sample

TrajectoryBufferSample(experience={'action': Array([[[2.06892246e-11, 9.55462814e-13, 3.47617985e-14, ...,
         1.70450963e-04, 9.51987028e-01, 4.78424989e-02],
        [2.23278240e-09, 9.99999881e-01, 6.94929483e-08, ...,
         2.38487264e-05, 9.99976158e-01, 7.72832406e-12],
        [9.99887466e-01, 2.59504759e-07, 4.24920762e-17, ...,
         2.51430203e-04, 9.93669868e-01, 6.07857388e-03],
        ...,
        [4.36003500e-09, 9.16709439e-07, 1.09520182e-02, ...,
         9.97643292e-01, 5.05451055e-04, 1.85125030e-03],
        [2.57591438e-03, 4.25381074e-07, 2.15260824e-03, ...,
         3.20033112e-04, 6.29408873e-07, 9.99679267e-01],
        [8.09437040e-07, 1.00822868e-02, 1.97056139e-07, ...,
         1.74851753e-02, 1.12450890e-01, 8.70063901e-01]],

       [[7.36846849e-02, 2.78365478e-04, 4.14273024e-01, ...,
         3.44924883e-05, 9.99965549e-01, 1.61586799e-10],
        [8.44443321e-07, 4.18407442e-09, 9.22161162e-01, ...,
         5.91705543e-07, 9.99983788e-0

In [8]:

def loss_fn_transformer(model: RubikTransformer, batch):
    state_logits, reward_value = model(batch["state_first"], batch["action"])

    # reshape state_logits
    # from (batch_size, sequence_length, 324) => (batch_size, sequence_length -1, 54, 6)
    state_logits = state_logits[:, 1:, :]
    state_logits = state_logits.reshape(
        (state_logits.shape[0], state_logits.shape[1], 54, 6)
    )

    reward_value = reward_value[:, 1:]

    loss_crossentropy = optax.softmax_cross_entropy_with_integer_labels(
        logits=state_logits, labels=batch["state_next"]
    ).mean()

    loss_reward = jnp.square(reward_value - batch["reward"]).mean()

    loss = loss_crossentropy + loss_reward

    return loss, (loss_crossentropy, loss_reward)

@nnx.jit
def train_step_transformer(
    model: RubikTransformer, optimizer: nnx.Optimizer, metrics: nnx.MultiMetric, batch
):
    """Train for a single step."""
    grad_fn = nnx.value_and_grad(loss_fn_transformer, has_aux=True)
    (loss, (loss_crossentropy, loss_reward)), grads = grad_fn(model, batch)
    metrics.update(
        loss=loss, loss_reward=loss_reward, loss_cross_entropy=loss_crossentropy
    )
    optimizer.update(grads)


In [9]:
key, subkey = jax.random.split(config.jax_key)
config.jax_key = key

buffer, buffer_list = dataset.fast_gathering_data(
    env,
    vmap_reset,
    vmap_step_proba,
    config.nb_games * 10,
    config.len_seq,
    buffer,
    buffer_list,
    subkey,
)


In [10]:

# transformer model calibration
for idx_step in tqdm(range(config.nb_step)):

    # training for world model
    key, subkey = jax.random.split(config.jax_key)
    config.jax_key = key

    if idx_step % config.add_data_every_step == 0:
        buffer, buffer_list = dataset.fast_gathering_data(
            env,
            vmap_reset,
            vmap_step_proba,
            int(config.nb_games / 10.),
            config.len_seq,
            buffer,
            buffer_list,
            config.jax_key,
        )

    sample = buffer.sample(buffer_list, subkey)
    sample = reshape_sample(sample)

    # we update the policy
    train_step_transformer(
        transformer, optimizer_worldmodel, metrics_train, sample.experience
    )

    if idx_step % config.log_every_step == 0:
        metrics_train_result = metrics_train.compute()
        print(metrics_train_result)

        wandb.log(metrics_train_result, step=idx_step)
        metrics_train.reset()

    if idx_step % config.log_eval_every_step == 0:

        key, subkey = jax.random.split(config.jax_key)
        config.jax_key = key
        
        buffer_eval, buffer_list_eval = dataset.fast_gathering_data(
            env,
            vmap_reset,
            vmap_step_proba,
            int(128),
            config.len_seq,
            buffer_eval,
            buffer_list_eval,
            subkey,
        )

        sample = buffer_eval.sample(buffer_list_eval, subkey)
        sample = reshape_sample(sample)

        loss, (loss_crossentropy, loss_reward) = loss_fn_transformer(transformer, sample.experience)

        metrics_eval.update(loss_eval=loss, loss_reward_eval=loss_reward, loss_cross_entropy_eval=loss_crossentropy)
        wandb.log(metrics_eval.compute(), step=idx_step)

        metrics_eval.reset()



  0%|          | 0/1000000 [00:00<?, ?it/s]

{'loss': Array(2.417289, dtype=float32), 'loss_reward': Array(0.3267387, dtype=float32), 'loss_cross_entropy': Array(2.0905504, dtype=float32)}


  0%|          | 9/1000000 [00:37<567:27:30,  2.04s/it]  

{'loss': Array(2.5006628, dtype=float32), 'loss_reward': Array(0.40404272, dtype=float32), 'loss_cross_entropy': Array(2.0966203, dtype=float32)}


  0%|          | 20/1000000 [00:39<137:42:36,  2.02it/s]

...

  0%|          | 4928/1000000 [16:09<39:36:53,  6.98it/s]

{'loss': Array(1.5163653, dtype=float32), 'loss_reward': Array(0.01005944, dtype=float32), 'loss_cross_entropy': Array(1.5063059, dtype=float32)}


  0%|          | 4939/1000000 [16:11<31:57:06,  8.65it/s]

{'loss': Array(1.5114313, dtype=float32), 'loss_reward': Array(0.01015394, dtype=float32), 'loss_cross_entropy': Array(1.5012774, dtype=float32)}


  0%|          | 4943/1000000 [16:12<54:06:47,  5.11it/s]

In [20]:
sample

TrajectoryBufferSample(experience={'action': Array([[[2.56099083e-05, 8.82284610e-08, 2.79197128e-08, ...,
         5.50557211e-10, 2.88656997e-06, 9.99997139e-01],
        [2.47293491e-10, 9.27802224e-10, 1.00000000e+00, ...,
         3.98532691e-04, 9.99601483e-01, 3.32171481e-08],
        [2.09763237e-14, 6.82480550e-06, 1.65303116e-09, ...,
         9.99999642e-01, 6.53804455e-10, 3.60816784e-07],
        ...,
        [1.27481584e-07, 9.99993205e-01, 4.13923908e-06, ...,
         6.99060142e-01, 2.78799862e-01, 2.21399572e-02],
        [1.02648977e-03, 3.72330985e-11, 9.40394122e-04, ...,
         1.11217632e-04, 7.39760087e-14, 9.99888778e-01],
        [1.46892916e-07, 2.50733981e-04, 3.89986113e-03, ...,
         3.50390064e-06, 9.99996543e-01, 2.23333588e-10]],

       [[2.48523785e-10, 3.07286285e-08, 1.07085234e-05, ...,
         9.99786496e-01, 2.13422667e-04, 2.66149840e-08],
        [4.37679797e-01, 9.33327512e-11, 1.15435640e-07, ...,
         2.28540314e-14, 7.32413685e-1

In [15]:
sample = buffer_eval.sample(buffer_list_eval, subkey)
sample = reshape_sample(sample)


In [16]:
loss, (loss_crossentropy, loss_reward) = loss_fn_transformer(transformer, sample.experience)


AttributeError: 'RubikTransformer' object has no attribute 'state_mapping'

In [15]:
# save buffer, buffer_list
# in pickle 
import pickle

state_weight = nnx.state(transformer)

In [16]:
state_weight

State({
  'action_mapping': {
    'bias': VariableState(
      type=Param,
      value=Array([ 0.0489077 ,  0.05748522, -0.04591497,  0.06825393, -0.01245873,
             -0.04504352, -0.02600798, -0.01612351,  0.04197448, -0.01106876,
              0.04133981,  0.00250468, -0.01917384,  0.08190358, -0.02015413,
              0.01493297,  0.07010985,  0.01872672,  0.00455121, -0.07804424,
             -0.02749256,  0.07241128, -0.01716232, -0.00095554, -0.07598419,
              0.07997464,  0.00542156,  0.0525552 , -0.02168683,  0.00650362,
             -0.05971894, -0.03304074,  0.01576223, -0.0412701 ,  0.07530559,
             -0.00969784, -0.03476293, -0.02465479,  0.01211519,  0.04951988,
              0.0479618 ,  0.02026099,  0.04587942, -0.00183689, -0.0394596 ,
              0.05025787,  0.03260148,  0.05018431,  0.03647119,  0.05557197,
              0.01052153, -0.07949464, -0.0741701 ,  0.01655396, -0.03202853,
             -0.01196375, -0.04284544, -0.04348039,  0.019043

In [17]:
# save state into pickle
with open('state_probainput_vscale2.pickle', 'wb') as handle:
    pickle.dump(state_weight, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
state_weight

State({})

In [13]:
nnx.state(transformer)

State({})

In [14]:

transformer

RubikTransformer()